# Milestone 4
**Name:** Eula Fullerton  
**Due Date:** Nov 3, 2024  
**Class:** DSC540-T302 Data Preparation  
**Professor:** Professor Williams   

## Connecting to an API/Pulling in the Data and Cleaning/Formatting


Perform at least 5 data transformation and/or cleansing steps to your API data. The below examples are not required - they are just potential transformations you could do. If your data doesn't work for these scenarios, complete different transformations. You can do the same transformation multiple times if needed to clean your data. The goal is a clean dataset at the end of the milestone. As a reminder - you cannot export your API data to CSV to work with it, you must do all the work directly against the API/JSON source.

Examples:
- Replace Headers
- Format data into a more readable format
- Identify outliers and bad data
- Find duplicates
- Fix casing or inconsistent values
- Conduct Fuzzy Matching
- Make sure you clearly number and label each transformation step (Step #1, Step #2, etc.) in your code and describe what it is doing in 1-2 sentences.

Each transformation should:
- be labeled with description or what it is doing.
- Human readable dataset after all transformations should be printed at the end of your notebook.
- 1 paragraph of the ethical implications of data wrangling specific to your datasource and the steps you completed answering the following questions:
- What changes were made to the data?
- Are there any legal or regulatory guidelines for your data or project topic?
- What risks could be created based on the transformations done?
- Did you make any assumptions in cleaning/transforming the data?
- How was your data sourced / verified for credibility?
- Was your data acquired in an ethical way?
- How would you mitigate any of the ethical implications you have identified?

## About Open FDA App

openFDA is an Elasticsearch-based API that serves public FDA data about nouns like drugs, devices, and foods.

Each of these nouns has one or more categories, which serve unique data-such as data about recall enforcement reports, or about adverse events. Every query to the API must go through one endpoint for one kind of data.

Not all data in openFDA has been validated for clinical or production use. And because openFDA only serves publicly available data, it does not contain data with Personally Identifiable Information about patients or other sensitive information.

"API" is an acronym for Application Programming Interface. An API call is any request sent to the API. Requests are typically sent to the API in one of two ways: 1. Manually using a web browser (such as navigating to the URL https://api.fda.gov/drug/label.json) or 2. Programmatically sending the request via executing code that sends the API call and processes the response. Continue reading this documentation for more details on how to compose an API call for openFDA specifically.

The API returns individual results as JSON by default. The JSON object has two sections:

meta: Metadata about the query, including a disclaimer, link to data license, last-updated date, and total matching records, if applicable.

results: An array of matching results, dependent on which endpoint was queried.

## Step 1: Connect to API

In [8]:
import requests

# API key
api_key = "LEwHbldDDJhHhHq3riKDswyLbOKo6HsRXHXG3NoL"

# Base URL
base_url = "https://api.fda.gov/drug/event.json"

# Drugs to search for with pregnancy relevance
drugs = [
    "Insulin", "Labetalol", "Magnesium sulfate", "Progesterone",
    "Ceftriaxone", "Penicillin G", "Azithromycin", "Lamivudine", "Nicotine"
]

# Dictionary to store results for each drug
drug_results = {}

# Loop through each drug and fetch pregnancy-relevant data
for drug in drugs:
    # Include pregnancy-related search terms with the drug name
    search_query = f'patient.drug.medicinalproduct:"{drug}" AND (patient.reaction.reactionmeddrapt:"pregnancy" OR patient.reaction.reactionmeddrapt:"pregnant" OR patient.reaction.reactionmeddrapt:"fetal")'
    url = f"{base_url}?api_key={api_key}&search={search_query}&limit=5"  # limit results for simplicity
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        drug_results[drug] = data if data.get("results") else None
        print(f"Pregnancy-related results found for {drug}")
    else:
        drug_results[drug] = None
        print(f"No results found for {drug}: {response.status_code} - {response.text}")

# write results into file to view
import json
with open("pregnancy_related_drug_results.json", "w") as file:
    json.dump(drug_results, file)

Pregnancy-related results found for Insulin
Pregnancy-related results found for Labetalol
Pregnancy-related results found for Magnesium sulfate
Pregnancy-related results found for Progesterone
Pregnancy-related results found for Ceftriaxone
Pregnancy-related results found for Penicillin G
Pregnancy-related results found for Azithromycin
Pregnancy-related results found for Lamivudine
Pregnancy-related results found for Nicotine


## Step 1: Load and Inspect Data

Load JSON file and inspect the structure to understand relevant fields

In [12]:
!pip install fuzzywuzzy

In [20]:
import json
import pandas as pd
from fuzzywuzzy import process  # For fuzzy matching
from IPython.display import display  # For displaying DataFrames in Jupyter Notebook

# Load JSON data
with open('pregnancy_related_drug_results.json', 'r') as file:
    data = json.load(file)

# Initial inspection of the structure
data.keys()

dict_keys(['Insulin', 'Labetalol', 'Magnesium sulfate', 'Progesterone', 'Ceftriaxone', 'Penicillin G', 'Azithromycin', 'Lamivudine', 'Nicotine'])

## Step 2: Flatten Data Structure

Extract nested fields such as patient, reaction, and drug into a flat structure to make it easier to analyze in a dataframe.

In [24]:
# Flatten JSON structure for DataFrame creation
reports = []
for drug, entries in data.items():
    if entries and "results" in entries:
        for result in entries["results"]:
            report = {
                "report_id": result.get("safetyreportid"),
                "patient_age": result.get("patient", {}).get("patientonsetage"),
                "patient_sex": result.get("patient", {}).get("patientsex"),
                "patient_weight": result.get("patient", {}).get("patientweight"),
                "reaction_event": [reaction.get("reactionmeddrapt") for reaction in result.get("patient", {}).get("reaction", [])],
                "medicinal_product": [drug_info.get("medicinalproduct") for drug_info in result.get("patient", {}).get("drug", [])],
                "drug_indication": [drug_info.get("drugindication") for drug_info in result.get("patient", {}).get("drug", [])],
                "seriousness_congenital_anomaly": result.get("seriousnesscongenitalanomali")
            }
            reports.append(report)

# Create DataFrame
df = pd.DataFrame(reports)
display(df.head())

,report_id,patient_age,patient_sex,patient_weight,reaction_event,medicinal_product,drug_indication,seriousness_congenital_anomaly
0,10009044,1,1,4,"[Congenital hydronephrosis, Ureteric stenosis,...","[METOPROLOL (UNKNOWN), BISOPROLOL, INSULIN, FO...","[HYPERTENSION, HYPERTENSION, GESTATIONAL DIABE...",1
1,10016807,None,0,None,"[Foetal death, Foetal heart rate deceleration ...","[HYDRALAZINE, METHYLDOPA, TELMISARTAN, AMLODIP...","[None, None, None, None, None, None, None, Non...",None
2,10019262,None,2,2.54,"[Foetal exposure during pregnancy, Multiple co...","[EFFEXOR, ZYPREXA, LEVOTHYROX, INSULIN]","[None, None, None, GESTATIONAL DIABETES]",1
3,10023714,35,2,68,"[No therapeutic response, Ectopic pregnancy, E...","[INSULIN GLARGINE, SOLOSTAR, HUMALOG]","[DIABETES MELLITUS, None, None]",None
4,10038823,0,2,3.6,"[Ventricular septal defect, Atrial septal defe...","[COMBIVIR, KALETRA, METFORMIN, INSULIN, NEXIUM...","[HIV INFECTION, HIV INFECTION, DIABETES MELLIT...",1


## Step 3: Standaradize Column Names

Standardize the column names and rename them all to lowercase and replace spaces with underscoress for consistency

In [28]:
# Standardize column names
df.columns = [col.lower().replace(" ", "_") for col in df.columns]
display(df.head())

,report_id,patient_age,patient_sex,patient_weight,reaction_event,medicinal_product,drug_indication,seriousness_congenital_anomaly
0,10009044,1,1,4,"[Congenital hydronephrosis, Ureteric stenosis,...","[METOPROLOL (UNKNOWN), BISOPROLOL, INSULIN, FO...","[HYPERTENSION, HYPERTENSION, GESTATIONAL DIABE...",1
1,10016807,None,0,None,"[Foetal death, Foetal heart rate deceleration ...","[HYDRALAZINE, METHYLDOPA, TELMISARTAN, AMLODIP...","[None, None, None, None, None, None, None, Non...",None
2,10019262,None,2,2.54,"[Foetal exposure during pregnancy, Multiple co...","[EFFEXOR, ZYPREXA, LEVOTHYROX, INSULIN]","[None, None, None, GESTATIONAL DIABETES]",1
3,10023714,35,2,68,"[No therapeutic response, Ectopic pregnancy, E...","[INSULIN GLARGINE, SOLOSTAR, HUMALOG]","[DIABETES MELLITUS, None, None]",None
4,10038823,0,2,3.6,"[Ventricular septal defect, Atrial septal defe...","[COMBIVIR, KALETRA, METFORMIN, INSULIN, NEXIUM...","[HIV INFECTION, HIV INFECTION, DIABETES MELLIT...",1


## Step 4: Flatten Lists in Columns

In [31]:
# Flatten lists in specified columns, handling None values
df['reaction_event'] = df['reaction_event'].apply(lambda x: ', '.join(filter(None, x)) if isinstance(x, list) else x)
df['medicinal_product'] = df['medicinal_product'].apply(lambda x: ', '.join(filter(None, x)) if isinstance(x, list) else x)
df['drug_indication'] = df['drug_indication'].apply(lambda x: ', '.join(filter(None, x)) if isinstance(x, list) else x)

# Display the modified DataFrame
display(df.head())

,report_id,patient_age,patient_sex,patient_weight,reaction_event,medicinal_product,drug_indication,seriousness_congenital_anomaly
0,10009044,1,1,4,"Congenital hydronephrosis, Ureteric stenosis, ...","METOPROLOL (UNKNOWN), BISOPROLOL, INSULIN, FOL...","HYPERTENSION, HYPERTENSION, GESTATIONAL DIABET...",1
1,10016807,None,0,None,"Foetal death, Foetal heart rate deceleration a...","HYDRALAZINE, METHYLDOPA, TELMISARTAN, AMLODIPI...",,None
2,10019262,None,2,2.54,"Foetal exposure during pregnancy, Multiple con...","EFFEXOR, ZYPREXA, LEVOTHYROX, INSULIN",GESTATIONAL DIABETES,1
3,10023714,35,2,68,"No therapeutic response, Ectopic pregnancy, Ex...","INSULIN GLARGINE, SOLOSTAR, HUMALOG",DIABETES MELLITUS,None
4,10038823,0,2,3.6,"Ventricular septal defect, Atrial septal defec...","COMBIVIR, KALETRA, METFORMIN, INSULIN, NEXIUM,...","HIV INFECTION, HIV INFECTION, DIABETES MELLITU...",1


## Step 5: Remove duplicate reports

Duplicates were removed from the report

In [49]:
# Identify duplicate entries based on 'report_id', excluding the first occurrence
duplicates = df[df.duplicated(subset=['report_id'], keep='first')]

# Display the duplicates
display(duplicates)

,report_id,patient_age,patient_sex,patient_weight,reaction_event,medicinal_product,drug_indication,seriousness_congenital_anomaly
38,10014483,8,None,None,"Anaemia, Toxicity to various agents, Foetal ex...","ISENTRESS, ISENTRESS, ISENTRESS, ABACAVIR, ABA...","HIV infection, Product used for unknown indica...",2


In [53]:
# Remove duplicate entries based on 'report_id'
df.drop_duplicates(subset=['report_id'], inplace=True)
display(df.head())

,report_id,patient_age,patient_sex,patient_weight,reaction_event,medicinal_product,drug_indication,seriousness_congenital_anomaly
0,10009044,1,1,4,"Congenital hydronephrosis, Ureteric stenosis, ...","METOPROLOL (UNKNOWN), BISOPROLOL, INSULIN, FOL...","HYPERTENSION, HYPERTENSION, GESTATIONAL DIABET...",1
1,10016807,None,0,None,"Foetal death, Foetal heart rate deceleration a...","HYDRALAZINE, METHYLDOPA, TELMISARTAN, AMLODIPI...",,None
2,10019262,None,2,2.54,"Foetal exposure during pregnancy, Multiple con...","EFFEXOR, ZYPREXA, LEVOTHYROX, INSULIN",GESTATIONAL DIABETES,1
3,10023714,35,2,68,"No therapeutic response, Ectopic pregnancy, Ex...","INSULIN GLARGINE, SOLOSTAR, HUMALOG",DIABETES MELLITUS,None
4,10038823,0,2,3.6,"Ventricular septal defect, Atrial septal defec...","COMBIVIR, KALETRA, METFORMIN, INSULIN, NEXIUM,...","HIV INFECTION, HIV INFECTION, DIABETES MELLITU...",1


## Step 7: Standardize Text Casing

convert medicinal_product and reaction event columns to lowercase to ensure consistent casing

In [57]:
# Standardize text casing
df['medicinal_product'] = df['medicinal_product'].str.lower()
df['reaction_event'] = df['reaction_event'].str.lower()
display(df[['medicinal_product', 'reaction_event']].head())

,medicinal_product,reaction_event
0,"metoprolol (unknown), bisoprolol, insulin, fol...","congenital hydronephrosis, ureteric stenosis, ..."
1,"hydralazine, methyldopa, telmisartan, amlodipi...","foetal death, foetal heart rate deceleration a..."
2,"effexor, zyprexa, levothyrox, insulin","foetal exposure during pregnancy, multiple con..."
3,"insulin glargine, solostar, humalog","no therapeutic response, ectopic pregnancy, ex..."
4,"combivir, kaletra, metformin, insulin, nexium,...","ventricular septal defect, atrial septal defec..."


## Step 8: Create a New Column for target drug

Separate the target drugs below from the others in the medicinal product column: 
["insulin", "labetalol", "magnesium sulfate", "progesterone", "ceftriaxone", "penicillin g", "azithromycin", "lamivudine", "nicotine"] 

In [61]:
# Define the target drugs (case-insensitive)
target_drugs = ["insulin", "labetalol", "magnesium sulfate", "progesterone", 
                "ceftriaxone", "penicillin g", "azithromycin", "lamivudine", "nicotine"]

# Function to separate target drugs from other products
def separate_drugs(row):
    products = [prod.strip().lower() for prod in row['medicinal_product'].split(',')]
    target = [prod for prod in products if any(td in prod for td in target_drugs)]
    other = [prod for prod in products if prod not in target]
    return pd.Series([', '.join(target), ', '.join(other)])

# Apply the function to split target and other drugs into separate columns
df[['target_medicinal_product', 'other_medicinal_product']] = df.apply(separate_drugs, axis=1)

# Display the DataFrame with separated columns
display(df[['medicinal_product', 'target_medicinal_product', 'other_medicinal_product', 'reaction_event']])

,medicinal_product,target_medicinal_product,other_medicinal_product,reaction_event
0,"metoprolol (unknown), bisoprolol, insulin, fol...",insulin,"metoprolol (unknown), bisoprolol, folio, dopeg...","congenital hydronephrosis, ureteric stenosis, ..."
1,"hydralazine, methyldopa, telmisartan, amlodipi...",insulin,"hydralazine, methyldopa, telmisartan, amlodipi...","foetal death, foetal heart rate deceleration a..."
2,"effexor, zyprexa, levothyrox, insulin",insulin,"effexor, zyprexa, levothyrox","foetal exposure during pregnancy, multiple con..."
3,"insulin glargine, solostar, humalog",insulin glargine,"solostar, humalog","no therapeutic response, ectopic pregnancy, ex..."
4,"combivir, kaletra, metformin, insulin, nexium,...",insulin,"combivir, kaletra, metformin, nexium, prenatal...","ventricular septal defect, atrial septal defec..."
5,"mycophenolate mofetil, calcitriol, azathioprin...",labetalol,"mycophenolate mofetil, calcitriol, azathioprin...","maternal exposure during pregnancy, foetal death"
6,"nifedipine, nifedipine, labetalol, labetalol","labetalol, labetalol","nifedipine, nifedipine","neonatal respiratory distress syndrome, jaundi..."
7,"nifedipine, nifedipine, labetalol",labetalol,"nifedipine, nifedipine","hypertension, premature delivery, exposure dur..."
8,"hydrochlorothiazide., triamcinolone, labetalol...",labetalol,"hydrochlorothiazide., triamcinolone, tylenol, ...","injury, premature delivery, device breakage, u..."
9,"oxytocin, salbutamol, fluticasone, calcium, la...",labetalol,"oxytocin, salbutamol, fluticasone, calcium, fe...","hypertension, renal failure chronic, condition..."


In [63]:
# Remove the original medicinal_product column afer separating the target and other products
df.drop(columns=['medicinal_product'], inplace=True)

# Display the updated DataFrame
display(df[['target_medicinal_product', 'other_medicinal_product', 'reaction_event']])


,target_medicinal_product,other_medicinal_product,reaction_event
0,insulin,"metoprolol (unknown), bisoprolol, folio, dopeg...","congenital hydronephrosis, ureteric stenosis, ..."
1,insulin,"hydralazine, methyldopa, telmisartan, amlodipi...","foetal death, foetal heart rate deceleration a..."
2,insulin,"effexor, zyprexa, levothyrox","foetal exposure during pregnancy, multiple con..."
3,insulin glargine,"solostar, humalog","no therapeutic response, ectopic pregnancy, ex..."
4,insulin,"combivir, kaletra, metformin, nexium, prenatal...","ventricular septal defect, atrial septal defec..."
5,labetalol,"mycophenolate mofetil, calcitriol, azathioprin...","maternal exposure during pregnancy, foetal death"
6,"labetalol, labetalol","nifedipine, nifedipine","neonatal respiratory distress syndrome, jaundi..."
7,labetalol,"nifedipine, nifedipine","hypertension, premature delivery, exposure dur..."
8,labetalol,"hydrochlorothiazide., triamcinolone, tylenol, ...","injury, premature delivery, device breakage, u..."
9,labetalol,"oxytocin, salbutamol, fluticasone, calcium, fe...","hypertension, renal failure chronic, condition..."


## Step 9: Create a new column for target pregnancy reactions

Separate pregnancy related reaction evens and other reaction events

In [67]:
# Define pregnancy-related keywords
pregnancy_keywords = ["pregnancy", "premature delivery", "foetal", "ectopic", "exposure during pregnancy"]

# Function to separate pregnancy-related reactions from other reactions
def separate_reactions(row):
    reactions = [reaction.strip().lower() for reaction in row['reaction_event'].split(',')]
    pregnancy_reactions = [reaction for reaction in reactions if any(keyword in reaction for keyword in pregnancy_keywords)]
    other_reactions = [reaction for reaction in reactions if reaction not in pregnancy_reactions]
    return pd.Series([', '.join(pregnancy_reactions), ', '.join(other_reactions)])

# Apply the function to split reactions into separate columns
df[['pregnancy_reaction_event', 'other_reaction_event']] = df.apply(separate_reactions, axis=1)

# Drop the original 'reaction_event' column 
df.drop(columns=['reaction_event'], inplace=True)

# Display the updated DataFrame
display(df[['target_medicinal_product', 'other_medicinal_product', 'pregnancy_reaction_event', 'other_reaction_event']])

,target_medicinal_product,other_medicinal_product,pregnancy_reaction_event,other_reaction_event
0,insulin,"metoprolol (unknown), bisoprolol, folio, dopeg...",foetal exposure during pregnancy,"congenital hydronephrosis, ureteric stenosis"
1,insulin,"hydralazine, methyldopa, telmisartan, amlodipi...","foetal death, foetal heart rate deceleration a...",premature baby
2,insulin,"effexor, zyprexa, levothyrox",foetal exposure during pregnancy,multiple congenital abnormalities
3,insulin glargine,"solostar, humalog","ectopic pregnancy, exposure during pregnancy",no therapeutic response
4,insulin,"combivir, kaletra, metformin, nexium, prenatal...",foetal exposure during pregnancy,"ventricular septal defect, atrial septal defec..."
5,labetalol,"mycophenolate mofetil, calcitriol, azathioprin...","maternal exposure during pregnancy, foetal death",
6,"labetalol, labetalol","nifedipine, nifedipine",foetal exposure during pregnancy,"neonatal respiratory distress syndrome, jaundi..."
7,labetalol,"nifedipine, nifedipine","premature delivery, exposure during pregnancy","hypertension, proteinuria, umbilical cord vasc..."
8,labetalol,"hydrochlorothiazide., triamcinolone, tylenol, ...","premature delivery, pregnancy with contracepti...","injury, device breakage, uterine perforation, ..."
9,labetalol,"oxytocin, salbutamol, fluticasone, calcium, fe...",maternal exposure during pregnancy,"hypertension, renal failure chronic, condition..."


## Download and display final data

In [70]:
# Save the final DataFrame to a CSV file
final_file_path = 'final_pregnancy_drug_data.csv'
df.to_csv(final_file_path, index=False)

# Display the final DataFrame
display(df)

# Provide a download link for the CSV file 
from IPython.display import FileLink
FileLink(final_file_path)

,report_id,patient_age,patient_sex,patient_weight,drug_indication,seriousness_congenital_anomaly,target_medicinal_product,other_medicinal_product,pregnancy_reaction_event,other_reaction_event
0,10009044,1,1,4,"HYPERTENSION, HYPERTENSION, GESTATIONAL DIABET...",1,insulin,"metoprolol (unknown), bisoprolol, folio, dopeg...",foetal exposure during pregnancy,"congenital hydronephrosis, ureteric stenosis"
1,10016807,None,0,None,,None,insulin,"hydralazine, methyldopa, telmisartan, amlodipi...","foetal death, foetal heart rate deceleration a...",premature baby
2,10019262,None,2,2.54,GESTATIONAL DIABETES,1,insulin,"effexor, zyprexa, levothyrox",foetal exposure during pregnancy,multiple congenital abnormalities
3,10023714,35,2,68,DIABETES MELLITUS,None,insulin glargine,"solostar, humalog","ectopic pregnancy, exposure during pregnancy",no therapeutic response
4,10038823,0,2,3.6,"HIV INFECTION, HIV INFECTION, DIABETES MELLITU...",1,insulin,"combivir, kaletra, metformin, nexium, prenatal...",foetal exposure during pregnancy,"ventricular septal defect, atrial septal defec..."
5,10017067,None,2,108.96,"RENAL TRANSPLANT, RENAL TRANSPLANT, RENAL TRAN...",None,labetalol,"mycophenolate mofetil, calcitriol, azathioprin...","maternal exposure during pregnancy, foetal death",
6,10027891,None,1,None,,None,"labetalol, labetalol","nifedipine, nifedipine",foetal exposure during pregnancy,"neonatal respiratory distress syndrome, jaundi..."
7,10027954,33,2,None,"PRE-ECLAMPSIA, HYPERTENSION",None,labetalol,"nifedipine, nifedipine","premature delivery, exposure during pregnancy","hypertension, proteinuria, umbilical cord vasc..."
8,10047438,39,2,None,CONTRACEPTION,None,labetalol,"hydrochlorothiazide., triamcinolone, tylenol, ...","premature delivery, pregnancy with contracepti...","injury, device breakage, uterine perforation, ..."
9,10051068,28,2,None,"LABOUR INDUCTION, ASTHMA, ASTHMA, GESTATIONAL ...",None,labetalol,"oxytocin, salbutamol, fluticasone, calcium, fe...",maternal exposure during pregnancy,"hypertension, renal failure chronic, condition..."


C:\Users\samue\Data Preparation\final_pregnancy_drug_data.csv

## Ethical Considertions of Data Wrangling

The changess made included handling missing values, removing duplicates, as well as creating and removing columns.  These steps improve data consistency and readability, however could impact the analysis's integridy by excluding or altering original data.  The source of the data is regulated by the FDA, which specifies privacy guidlines to protect patient confidentialiity. Data analysis must comply with FDA terms since the information gathered is senstive health-related data.  The FDA also advises caution to the open API, since the data ins unvalidated.  Ethical considerations and documentation of each transformation step in the data wrangling process were performed to ensure transparancy to the potential data loss or introduction of incomplete or biased results.